<a href="https://colab.research.google.com/github/grace-arina/Diagnosing-Pneumonia_CNN/blob/Grace_starter-models/Grace_starter_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import opendatasets as od
od.download('https://www.kaggle.com/paultimothymooney/chest-xray-pneumonia')

Skipping, found downloaded files in "./chest-xray-pneumonia" (use force=True to force download)


In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Data should be formatted into appropriately preprocessed floatingpoint
tensors before being fed into the network. Currently, the data sits on a drive as
JPEG files, so the steps for getting it into the network are roughly as follows:
1.  Read the picture files.
2.  Decode the JPEG content to RGB grids of pixels.
3.  Convert these into floating-point tensors.
4.  Rescale the pixel values (between 0 and 255) to the [0, 1] interval (as you know,
neural networks prefer to deal with small input values).

In [4]:
# Creating an datagenerator object that will perform image augmentation
train_datagen = ImageDataGenerator(rescale=1./255,
                                   rotation_range=90,
                                   width_shift_range=.2,
                                   height_shift_range=.2,
                                   shear_range=.2,
                                   zoom_range=.2,
                                   horizontal_flip=True,
                                   brightness_range=[.5, 1.5])

# Datagenerators for the test and validation set will only rescale the images
test_datagen = ImageDataGenerator(rescale=1./255)
val_datagen = ImageDataGenerator(rescale=1./255)


# Applying the datagenerator objects to the images in the folders

train_set = train_datagen.flow_from_directory('./chest-xray-pneumonia/chest_xray/train/',
                                                 target_size=(150, 150),
                                                 batch_size=20,
                                                 class_mode='binary',
                                                 color_mode='grayscale')

val_set = val_datagen.flow_from_directory('./chest-xray-pneumonia/chest_xray/val/',
                                          target_size=(150, 150),
                                          batch_size=20,
                                          class_mode='binary',
                                          color_mode='grayscale')

test_set = test_datagen.flow_from_directory('./chest-xray-pneumonia/chest_xray/test/',
                                            target_size=(150, 150),
                                            batch_size=20,
                                            class_mode='binary', 
                                            color_mode='grayscale')

Found 5216 images belonging to 2 classes.
Found 16 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


Each split contains the same number of samples from each class: this is a balanced
binary-classification problem, which means classification accuracy will be an
appropriate measure of success.

In [10]:
from keras import layers
from keras import models
model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu',
input_shape=(150, 150, 1)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(128, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Flatten())
model.add(layers.Dense(512, activation='relu'))
model.add(layers.Dense(1, activation='sigmoid'))

Let’s look at how the dimensions of the feature maps change with every successive
layer:

In [11]:
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_4 (Conv2D)           (None, 148, 148, 32)      320       
                                                                 
 max_pooling2d_4 (MaxPooling  (None, 74, 74, 32)       0         
 2D)                                                             
                                                                 
 conv2d_5 (Conv2D)           (None, 72, 72, 64)        18496     
                                                                 
 max_pooling2d_5 (MaxPooling  (None, 36, 36, 64)       0         
 2D)                                                             
                                                                 
 conv2d_6 (Conv2D)           (None, 34, 34, 128)       73856     
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 17, 17, 128)     

For the compilation step, I’ll go with the RMSprop optimizer because I
ended the network with a single sigmoid unit, I’ll use binary crossentropy as the
loss

In [12]:
from keras import optimizers
model.compile(loss='binary_crossentropy',
optimizer='rmsprop',
metrics=['acc'])

Let’s fit the model to the data using the generator.

In [ ]:
history = model.fit(train_set,
                    steps_per_epoch=100,
                    epochs=30, 
                    validation_data=val_set,
                    validation_steps=50)

Epoch 1/30
100/100 [==============================] - 108s 1s/step - loss: 0.5149 - acc: 0.7415
Epoch 2/30
100/100 [==============================] - 112s 1s/step - loss: 0.4798 - acc: 0.7735
Epoch 3/30
100/100 [==============================] - 111s 1s/step - loss: 0.4748 - acc: 0.7796
Epoch 4/30
100/100 [==============================] - 108s 1s/step - loss: 0.4246 - acc: 0.8025
Epoch 5/30
100/100 [==============================] - 109s 1s/step - loss: 0.4090 - acc: 0.8150
Epoch 6/30
100/100 [==============================] - 112s 1s/step - loss: 0.3974 - acc: 0.8150
Epoch 7/30
100/100 [==============================] - 112s 1s/step - loss: 0.3870 - acc: 0.8181
Epoch 8/30
100/100 [==============================] - 109s 1s/step - loss: 0.3551 - acc: 0.8385
Epoch 9/30
100/100 [==============================] - 107s 1s/step - loss: 0.3786 - acc: 0.8405
Epoch 10/30
100/100 [==============================] - 112s 1s/step - loss: 0.3718 - acc: 0.8360
Epoch 11/30
100/100 [==================

It's good practice to save the model after training

In [ ]:
model.save('baseline_model.h5')

Let’s plot the loss and accuracy of the model over the training and validation data
during training

In [ ]:
acc = history.history['acc']
val_acc = history.history['val_acc']
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(acc) + 1)
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.legend()
plt.figure()
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.legend()
plt.show()